In [1]:
print("Hello world!")

Hello world!


In [2]:
from pyspark.sql import functions as f

In [3]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [34]:
from pyspark.sql.types import StructType as struct, IntegerType, StringType as String,TimestampType as Timestamp,StructType,StructField, DoubleType,DateType

In [42]:
#json_sdf = spark.readStream.json("data/json_files/")
#spark.sql("set spark.sql.streaming.schemaInference=true")

json_sdf = spark.readStream.json("./data/json_files/")




In [ ]:
json_sdf.isStreaming

In [16]:
json_sdf.printSchema()

root
 |-- analytics: struct (nullable = true)
 |    |-- clicks: long (nullable = true)
 |    |-- impressions: long (nullable = true)
 |-- datetime: string (nullable = true)
 |-- sales: struct (nullable = true)
 |    |-- quantity: long (nullable = true)
 |    |-- total_price: double (nullable = true)



In [43]:
flat_df = json_sdf.select("analytics.*", "datetime", "sales.*")

flat_df.printSchema()


root
 |-- clicks: long (nullable = true)
 |-- impressions: long (nullable = true)
 |-- datetime: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- total_price: double (nullable = true)



In [44]:
                        
schema =           StructType(
                [
                    StructField("clicks", IntegerType(), True),
                    StructField("quantity", IntegerType(), True),
                    StructField("datetime", Timestamp(), True),
                    StructField("impressions", IntegerType(), True),
                    StructField("total_price", DoubleType(), True)
                ]
            )
schema


StructType(List(StructField(clicks,IntegerType,true),StructField(quantity,IntegerType,true),StructField(datetime,TimestampType,true),StructField(impressions,IntegerType,true),StructField(total_price,DoubleType,true)))

In [45]:
write_to = flat_df.writeStream.option("checkpointLocation","checkpoints")\
        .option("header", "true")\
        .outputMode("append")\
        .format("csv")\
        .start("data/csv_files/")\
    .awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/spark/python/lib/py4j-0.10.9.3-src.zip/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
write_to.isActive


In [ ]:
csv_sdf = spark.readStream \
          .format('csv') \
          .option('header',True) \
          .schema(schema) \
          .load("data/csv_files/")
                
                
                
        #csv(path = "data/csv_files/", header = True, inferSchema = True)
                

In [ ]:
tumblingWindows = csv_sdf.withWatermark("datetime", "5 minutes") \
                .groupBy(
                f.window(f.col("datetime"), "5 minutes")).agg(f.sum("clicks").alias("aggregate_clicks"), 
                f.sum("impressions").alias("aggregate_impressions"), 
                f.sum("quantity").alias("aggregate_quantity"), 
                f.sum("total_price").alias("aggregate_tprice"))
        
#tumblingWindows.show(truncate = False)


In [ ]:
ready_data = tumblingWindows.writeStream.option("checkpointLocation","checkpoints1")\
        .option("header", "true")\
        .outputMode("append")\
        .format("parquet")\
        .start("data/parquet_files/")\
    .awaitTermination()

In [46]:
import pandas as pd
pdf = pd.read_parquet('data/parquet_files', engine='pyarrow')

pdf.head()

,window,aggregate_clicks,aggregate_impressions,aggregate_quantity,aggregate_tprice
0,"{'start': 1657744500000000000, 'end': 16577448...",3217,3395,9068,3005.7
